In [18]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Set up executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

# 1. we're searching for elements with a specific combination of tag (div) and attribute (list_text)
# 2.  we're also telling our browser to wait one second before searching for components

[WDM] - ====== WebDriver manager ======


2022-07-13 11:39:51,979 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-13 11:39:52,102 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip


2022-07-13 11:39:52,297 INFO About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/mariasevillano/.wdm/drivers/chromedriver/mac64/103.0.5060.53]


2022-07-13 11:39:52,952 INFO Driver has been saved in cache [/Users/mariasevillano/.wdm/drivers/chromedriver/mac64/103.0.5060.53]


True

In [3]:
# we'll set up the HTML parser:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text') #This is our parent element. This means that this element holds all of the other elements within it, 
# and we'll reference it when we want to filter search results even further


In [4]:
# Scrape article's title
slide_elem.find('div', class_='content_title') # The output should be the HTML containing 
# the content title and anything else nested inside of that

<div class="content_title">NASA's Briefcase-Size MarCO Satellite Picks Up Honors</div>

In [5]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

#.get_text(). When this new method is chained onto .find(), only the text of the element is returned

"NASA's Briefcase-Size MarCO Satellite Picks Up Honors"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.'

### Image Scraping

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [14]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [15]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

ElementClickInterceptedException: Message: element click intercepted: Element <button class="btn btn-outline-light">...</button> is not clickable at point (317, 284). Other element would receive the click: <img class="fancybox-image" src="image/featured/mars2.jpg" alt="">
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x00000001067ab079 chromedriver + 4444281
1   chromedriver                        0x0000000106737403 chromedriver + 3970051
2   chromedriver                        0x00000001063d2038 chromedriver + 409656
3   chromedriver                        0x000000010640f9e5 chromedriver + 661989
4   chromedriver                        0x000000010640d593 chromedriver + 652691
5   chromedriver                        0x000000010640ac24 chromedriver + 642084
6   chromedriver                        0x0000000106409885 chromedriver + 637061
7   chromedriver                        0x00000001063fd6a9 chromedriver + 587433
8   chromedriver         

In [16]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [17]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [21]:
# read_html() specifically searches for and returns a list of tables found in the HTML
# By specifying an index of 0, we're telling Pandas to pull only the first table it encounters,
df = pd.read_html('https://galaxyfacts-mars.com')[0]
# we assign columns to the new DataFrame for additional clarity.
df.columns=['description', 'Mars', 'Earth']
#we're turning the Description column into the DataFrame's index. 
# inplace=True means that the updated index will remain in place
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [22]:
# convert our DataFrame back into HTML-ready code using the .to_html() 
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [23]:
# end session
browser.quit()